# Expressibility in quantum kernels 

In [3]:
import sys
import os
# to import quask, move from docs/source/notebooks to src
sys.path.append('../../../src')

In this tutorial, we start to charatetize the properties of a quantum kernel. The *expressibility* is definitively the most important. _quask_ has some built in facilities to evaluate a quantum kernel from this perspective. 

## Expressibility of a parameterized quantum circuit

The concept of _expressibility_ for $U(\mathbf{x})$ can be understood as its capacity to distribute classical data throughout the Hilbert space of the quantum system. To formally quantify this expressibility, we rely on the norm of the super-operator $A$, given by $||A||=\mathrm{Trace}[AA^\dagger]$. This can be defined as:

$$
A = \int_\text{Haar} (\left|\phi\right>\left<\phi\right|)^{\otimes t} d\phi - \int_\Theta (U(\mathbf{\theta}) \left|0\right>\left<0\right| U^\dagger (\mathbf{\theta}))^{\otimes t} d\mathbf{\theta},
$$

where $t$ is an integer with $t \ge 2$ \[sim19\].

The super-operator $A$ quantifies the extent to which the ensemble of states, obtained by initiating the system in $\ket{0}$ and evolving it with $U(\theta)$ (where $\theta$ is randomly chosen), deviates from the Haar-random ensemble of states. The Haar-random ensemble represents a uniform distribution of quantum states throughout the Hilbert space. When this deviation is small, we consider the unitary $U$ to be expressible.

As comparing a distribution to the true Haar-random distribution of states can be challenging, the concept of $\varepsilon$-approximate state $t$-design is employed. This ensemble of states closely approximates the Haar random ensemble of states up to the $t$-th statistical moment, hence the parameter $t$. For $t = 1$, only the first statistical moment (the average) is considered, which is generally less informative. For $t=2$, both the average and standard deviation are taken into account, making it a suitable choice for most use cases. The higher the value of $t$, the more precise the quantification of the deviation, but the more computationally expensive the calculation becomes.

Understanding the expressibility of the parameterized quantum circuit $U$ provides insights into the potential performance of the kernel that utilizes $U$. Further details on these aspects are covered in the tutorial [Spectral Bias]("../tutorial_quantum/quantum_3_spectralbias.html").


### The `KernelEvaluator` object

In _quask_, you can assess a kernel object based on specific criteria using the `KernelEvaluator`. Each sub-class of KernelEvaluator contains a single method, `evaluate`, which provides a cost associated with the given object. The kernel's quality with respect to a particular criterion is indicated by a lower cost. Additionally, the cost may be influenced by a set of data, as is the case with accuracy. More about the evaluators implemented in quask can be found in the tutorial [Quask Evaluators]("../tutorial_quantum/quantum_3_spectralbias.html").

### The `HaarEvaluator` object

The `HaarEvaluator` class inherits `KernelEvaluator` and is used to calculate $||A||$. It can be used as follows, 

In [7]:
from quask.core import Ansatz, Kernel, KernelFactory, KernelType

print(KernelFactory._KernelFactory__implementations)

{}


We see there are no implementations. First, we need to choose the SDK to use a specific `core_implementation` kernel, let's say Pennylane. As we did in the previous tutorial:

In [16]:
from quask.core_implementation import PennylaneKernel

def create_pennylane_noiseless(ansatz: Ansatz, measurement: str, type: KernelType):
    return PennylaneKernel(ansatz, measurement, type, device_name="default.qubit", n_shots=None)

KernelFactory.add_implementation('pennylane_noiseless', create_pennylane_noiseless)
KernelFactory.set_current_implementation('pennylane_noiseless')
print(KernelFactory._KernelFactory__implementations)
print(KernelFactory._KernelFactory__current_implementation)

{'pennylane_noiseless': <function create_pennylane_noiseless at 0x1503f9d00>}
pennylane_noiseless


In [17]:
from quask.evaluator import HaarEvaluator

N_FEATURES = 2
N_OPERATIONS = 3
N_QUBITS = 2
ansatz = Ansatz(n_features=N_FEATURES, n_qubits=N_QUBITS, n_operations=N_OPERATIONS)
ansatz.initialize_to_identity()
ansatz.change_operation(0, new_feature=0, new_wires=[0, 1], new_generator="ZZ", new_bandwidth=1.0)
ansatz.change_operation(1, new_feature=1, new_wires=[0, 1], new_generator="XX", new_bandwidth=1.0)
ansatz.change_operation(2, new_feature=2, new_wires=[0, 1], new_generator="IX", new_bandwidth=0.123)
kernel = KernelFactory.create_kernel(ansatz, "Z" * N_QUBITS, KernelType.FIDELITY)

he = HaarEvaluator(n_bins=40, n_samples=10000)
cost = he.evaluate(kernel=kernel, K=None, X=None, y=None)
print(f"Cost (norm of A): {cost:3.5f}")

Cost (norm of A): 0.16832


Usually, the kernel evaluator needs only the `kernel` argument. If the criteria depend on the data, `X` and `y` parameters must be provided, which correspond to the dataset features and dataset labels, respectively. If the kernel Gram matrix must be calculated, it will be done using `kernel` and `X`; however, if it has been pre-calculated previously, then it should be passed to the `K` parameter to avoid losing time to re-calculate it.


## Dynamical Lie algebra of a parameterized quantum circuit

The parameterized unitary transformation $U(\cdot)$, which operates on $n$ qubits, can be expressed as a sum of Pauli generators:

$$U(\mathbf{x}) = \exp\left(-i \sum_{j = 0}^{4^n-1} f_j(\mathbf{x}) \sigma_j \right)$$

Here, $f_j(\mathbf{x})$ belongs to $\mathbb{R}$, and $\sigma_j$ represents the tensor product of $n$ Pauli matrices. For instance:

- $\sigma_0 = \mathrm{Id} \otimes \mathrm{Id} \otimes ... \otimes \mathrm{Id}$,
- $\sigma_1 = \mathrm{Id} \otimes \mathrm{Id} \otimes ... \otimes X$,
- $\sigma_{4^n-1} = Z \otimes Z \otimes ... \otimes Z$.

Each unitary transformation can depend on all $4^n$ generators or only on a subset of them. In the context of a quantum circuit, $U$ is represented by the product of elementary one-qubit and two-qubit gates. To determine the generator of this transformation, we employ the tools of the Dynamical Lie Algebra, a vector space with a bilinear operation denoted as the commutator ($[\sigma, \eta] = \sigma\eta - \eta\sigma$). This vector space is spanned by the generators of $U$, which include the generators of both single one- and two-qubit gates and the generators derived through repeated application of the commutator until a fixed point is reached.

The number of generators is the rank of the Dynamical Lie Algebra. 

### The `LieRankEvaluator`

The rank of the Dynamical Lie Algebra can be determined using the `LieRankEvaluator`. This class inherits from `KernelEvaluator` and is employed in a manner similar to the previous examples. It's important to note that from a computational perspective, exact calculation is feasible only for small values of $n$ or specific cases. In the worst-case scenario, calculating an exponential number of commutations is required to cover all the $4^n$ potential Pauli strings. To address this, the search can be truncated once a predefined threshold $T$ is reached.

In [18]:
from quask.evaluator import LieRankEvaluator
lre = LieRankEvaluator(T=500)
cost = lre.evaluate(kernel=kernel, K=None, X=None, y=None)
print(f"Cost (-1 * rank of DLA): {cost:3.5f}")

Cost (-1 * rank of DLA): -8.00000


### Use of the DLA in quantum kernels

The application of the Dynamical Lie Algebra (DLA) in quantum machine learning has significantly contributed to the theoretical development of various tools. It has been demonstrated that the rank of the DLA serves as a proxy for expressibility. Essentially, the more generators a unitary transformation $U$ possesses, the greater the capacity to map quantum states across the Hilbert space of the quantum system \[lar21\].

However, it's worth noting that this measure lacks some precision. For instance, it doesn't account for the density of the distribution of quantum states, a consideration addressed by the norm of the super-operator $A$. Moreover, when we introduce a bandwidth parameter $\beta$ to restrict the rotational angles, we effectively limit the region in which states can be mapped. In such cases, we may encounter a unitary transformation that, despite having an exponential number of generators, exhibits only mild expressibility.

In [20]:
# let us build our Ansatz to test this hypothesis
# we can define a new function to construct the Ansatz since we will loop for different bandwidth values
def VaryingBandwidthKernel(N_FEATURES, N_QUBITS, N_OPERATIONS, bandwidth):
    ansatz = Ansatz(n_features=N_FEATURES, n_qubits=N_QUBITS, n_operations=N_OPERATIONS)
    ansatz.initialize_to_identity()
    # we choose a full entanglement setting
    ansatz.change_operation(0, new_feature=0, new_wires=[0, 1], new_generator="XX", new_bandwidth=bandwidth)
    ansatz.change_operation(1, new_feature=1, new_wires=[0, 2], new_generator="XX", new_bandwidth=bandwidth)
    ansatz.change_operation(2, new_feature=2, new_wires=[1, 2], new_generator="XX", new_bandwidth=bandwidth)
    kernel = KernelFactory.create_kernel(ansatz, "Z" * N_QUBITS, KernelType.FIDELITY)
    return kernel

In [22]:
import numpy as np
# parameters
N_FEATURES = 2
N_OPERATIONS = 3
N_QUBITS = 3

# let us sweep for different values of the bandwidth
bandwidths = np.linspace(1e-3,1e+0, 10).tolist()
bandwidths.reverse()

# we compute both the Haar Evaluator and the Lie Rank Evaluator
for beta in bandwidths:
    print(f"Bandwith: {beta:3.5f}")
    kernel = VaryingBandwidthKernel(N_FEATURES=N_FEATURES, N_QUBITS=N_QUBITS, N_OPERATIONS=N_OPERATIONS, bandwidth=beta)

    he = HaarEvaluator(n_bins=40, n_samples=10000)
    cost = he.evaluate(kernel=kernel, K=None, X=None, y=None)
    print(f"Cost (norm of A): {cost:3.5f}")
    lre = LieRankEvaluator(T=500)
    cost = lre.evaluate(kernel=kernel, K=None, X=None, y=None)
    print(f"Cost (-1 * rank of DLA): {cost:3.5f}")
    print(f"=====================================")

Bandwith: 1.00000
Cost (norm of A): 0.17330
Cost (-1 * rank of DLA): -4.00000
Bandwith: 0.88900
Cost (norm of A): 0.17907
Cost (-1 * rank of DLA): -4.00000
Bandwith: 0.77800
Cost (norm of A): 0.17554
Cost (-1 * rank of DLA): -4.00000
Bandwith: 0.66700
Cost (norm of A): 0.16823
Cost (-1 * rank of DLA): -4.00000
Bandwith: 0.55600
Cost (norm of A): 0.16663
Cost (-1 * rank of DLA): -4.00000
Bandwith: 0.44500
Cost (norm of A): 0.16426
Cost (-1 * rank of DLA): -4.00000
Bandwith: 0.33400
Cost (norm of A): 0.21903
Cost (-1 * rank of DLA): -4.00000
Bandwith: 0.22300
Cost (norm of A): 0.32297
Cost (-1 * rank of DLA): -4.00000
Bandwith: 0.11200
Cost (norm of A): 0.44571
Cost (-1 * rank of DLA): -4.00000
Bandwith: 0.00100
Cost (norm of A): 1.04596
Cost (-1 * rank of DLA): -4.00000


Indeed, we observe that at some point (in this case for $\beta_c = 0.334$) there is an inversion in the trend of the Haar cost. If decreasing the bandwith was having a positive effect on the expressibility for $\beta > \beta_c$ , once we cross this critical value, we are limiting the expressive power of our Ansatz. On the contrary, the `LieRankEvaluator` is not able to capture this aspect.

The rank of the DLA also sheds light on another intriguing aspect. Some relatively simple quantum circuits can be efficiently simulated on classical computers, rendering the use of quantum hardware redundant. This is particularly evident for circuits consisting solely of single-qubit gates. \[som06\] has established that unitary transformations with a polynomial number of generators can be efficiently simulated in polynomial time on classical hardware. While the reverse is not universally proven, having a multitude of generators offers favorable evidence that can be used to speculate that the chosen quantum circuit is challenging to simulate classically.

## References 

\[sim19\] Sim, Sukin, Peter D. Johnson, and Alán Aspuru‐Guzik. "Expressibility and entangling capability of parameterized quantum circuits for hybrid quantum‐classical algorithms." Advanced Quantum Technologies 2.12 (2019): 1900070.

\[lar21\] Larocca, Martin, et al. "Diagnosing barren plateaus with tools from quantum optimal control." Quantum 6 (2022): 824.

\[som06\] Somma, Rolando, et al. "Efficient solvability of Hamiltonians and limits on the power of some quantum computational models." Physical review letters 97.19 (2006): 190501.